<a href="https://colab.research.google.com/github/dariiimo/nlp2021/blob/main/lab4/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/нлп/'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [8]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sentences = DARINA_file.read().split('. ')[:-1]


max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens


RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [9]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


#### Tokenization&Embedding

In [10]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [11]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[-3.34585682e-02,  2.19931938e-02,  3.51932384e-02, ...,
         -1.09933130e-02,  1.73860677e-02, -6.33150339e-03],
        [ 5.09514660e-03, -4.65407036e-02,  3.56550477e-02, ...,
         -1.81864724e-02, -4.93431352e-02,  2.93092839e-02],
        [-3.91652472e-02, -2.60280501e-02,  2.39366628e-02, ...,
         -2.06253417e-02, -2.71889213e-02,  3.66695188e-02],
        ...,
        [ 1.23544335e-02,  4.77907173e-02, -2.54435297e-02, ...,
         -6.12269714e-03, -8.67024064e-05, -1.51677951e-02],
        [-2.47760657e-02,  1.09596029e-02, -7.10070133e-03, ...,
         -1.80762038e-02, -1.73626095e-03, -3.05696018e-02],
        [ 1.27626397e-02, -2.70363335e-02, -4.93188277e-02, ...,
         -3.70488539e-02,  4.61219810e-02, -4.66816798e-02]],

       [[-7.47419894e-04, -1.97347049e-02, -4.85479943e-02, ...,
         -3.62601168e-02,  1.22018829e-02,  1.05051510e-02],
        [-3.75435948e-02,  4.50884737e-02,  3.2

#### Train/Test Split

In [12]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
106,“If an agent can continually adjust its behavi...,Reinforcement Learning,0
104,"Based on the feedback it receives, the agent l...",Reinforcement Learning,0
115,Whereas existing baselines require human exper...,Reinforcement Learning,0
71,The agent (another name for reinforcement lear...,Reinforcement Learning,0
120,With facial recognition one can take as many f...,Neural network module for recognizing brain di...,1
...,...,...,...
25,"On a theoretical level, a policy is a mapping ...",Reinforcement Learning,0
96,Analysts first thought the move was a mistake ...,Reinforcement Learning,0
72,deepsense.ai took part in the “Learning to run...,Reinforcement Learning,0
194,"Clarified guidelines from the FDA, however, c...",Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 90.91%


### LSTM

In [15]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_lstm = tf.keras.Model(inputs, outputs, name="model_LSTM")


In [16]:
model_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [17]:
model_lstm.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [18]:
model_lstm_history = model_lstm.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=4,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/4
6/6 [==============================] - 6s 145ms/step - loss: 0.6922 - accuracy: 0.5174 - val_loss: 0.6886 - val_accuracy: 0.5909
Epoch 2/4
6/6 [==============================] - 0s 20ms/step - loss: 0.6805 - accuracy: 0.7209 - val_loss: 0.6824 - val_accuracy: 0.6136
Epoch 3/4
6/6 [==============================] - 0s 18ms/step - loss: 0.6556 - accuracy: 0.7907 - val_loss: 0.6617 - val_accuracy: 0.7045
Epoch 4/4
6/6 [==============================] - 0s 18ms/step - loss: 0.5732 - accuracy: 0.8953 - val_loss: 0.5599 - val_accuracy: 0.8182


In [19]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)


{'accuracy': 81.81818181818183,
 'f1': 0.8181818181818182,
 'precision': 0.8181818181818182,
 'recall': 0.8181818181818182}

### GRU

In [20]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x) 
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_gru = tf.keras.Model(inputs, outputs, name="model_GRU")


In [21]:
model_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [22]:
model_gru_history = model_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=5,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/5
6/6 [==============================] - 3s 123ms/step - loss: 0.6901 - accuracy: 0.5116 - val_loss: 0.6868 - val_accuracy: 0.5455
Epoch 2/5
6/6 [==============================] - 0s 18ms/step - loss: 0.6684 - accuracy: 0.6744 - val_loss: 0.6803 - val_accuracy: 0.5455
Epoch 3/5
6/6 [==============================] - 0s 18ms/step - loss: 0.6457 - accuracy: 0.7384 - val_loss: 0.6756 - val_accuracy: 0.5682
Epoch 4/5
6/6 [==============================] - 0s 19ms/step - loss: 0.6031 - accuracy: 0.7674 - val_loss: 0.6579 - val_accuracy: 0.6136
Epoch 5/5
6/6 [==============================] - 0s 18ms/step - loss: 0.5054 - accuracy: 0.8256 - val_loss: 0.5847 - val_accuracy: 0.7273


In [23]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 72.72727272727273,
 'f1': 0.7255502392344497,
 'precision': 0.7268842268842269,
 'recall': 0.7272727272727273}

### Bidirectional LSTM

In [24]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_lstm = tf.keras.Model(inputs, outputs, name="model_Bi_lstm")


In [25]:
model_bi_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [26]:
model_bi_lstm.summary()


Model: "model_Bi_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [27]:
model_bi_lstm_history = model_bi_lstm.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=4,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/4
6/6 [==============================] - 5s 215ms/step - loss: 0.6738 - accuracy: 0.7849 - val_loss: 0.6635 - val_accuracy: 0.7045
Epoch 2/4
6/6 [==============================] - 0s 30ms/step - loss: 0.5979 - accuracy: 0.8430 - val_loss: 0.6131 - val_accuracy: 0.7045
Epoch 3/4
6/6 [==============================] - 0s 27ms/step - loss: 0.4472 - accuracy: 0.8837 - val_loss: 0.5223 - val_accuracy: 0.7500
Epoch 4/4
6/6 [==============================] - 0s 25ms/step - loss: 0.1966 - accuracy: 0.9419 - val_loss: 0.3281 - val_accuracy: 0.8636


### Bidirectional GRU

In [28]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_gru = tf.keras.Model(inputs, outputs, name="model_Bi_gru")

In [29]:
model_bi_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [30]:
model_bi_gru.summary()

Model: "model_Bi_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,354,625
No

In [31]:
model_bi_gru_history = model_bi_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=10,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/10
6/6 [==============================] - 5s 245ms/step - loss: 0.6756 - accuracy: 0.6337 - val_loss: 0.6662 - val_accuracy: 0.5909
Epoch 2/10
6/6 [==============================] - 0s 26ms/step - loss: 0.6076 - accuracy: 0.7907 - val_loss: 0.6352 - val_accuracy: 0.6364
Epoch 3/10
6/6 [==============================] - 0s 24ms/step - loss: 0.5148 - accuracy: 0.8314 - val_loss: 0.5809 - val_accuracy: 0.7045
Epoch 4/10
6/6 [==============================] - 0s 25ms/step - loss: 0.3545 - accuracy: 0.8953 - val_loss: 0.4583 - val_accuracy: 0.8182
Epoch 5/10
6/6 [==============================] - 0s 25ms/step - loss: 0.1303 - accuracy: 1.0000 - val_loss: 0.3458 - val_accuracy: 0.8409
Epoch 6/10
6/6 [==============================] - 0s 24ms/step - loss: 0.0200 - accuracy: 1.0000 - val_loss: 0.4165 - val_accuracy: 0.8636
Epoch 7/10
6/6 [==============================] - 0s 25ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.5999 - val_accuracy: 0.8409
Epoch 8/10
6/6 [==========

## Здійснити порівняння ефективності моделей

### Naive Bayes

In [32]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 90.9090909090909,
 'f1': 0.9075369075369075,
 'precision': 0.922077922077922,
 'recall': 0.9090909090909091}

### LSTM

In [33]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)

{'accuracy': 90.9090909090909,
 'f1': 0.9090909090909091,
 'precision': 0.9090909090909091,
 'recall': 0.9090909090909091}

### GRU

In [34]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 77.27272727272727,
 'f1': 0.7727272727272727,
 'precision': 0.7856060606060605,
 'recall': 0.7727272727272727}

### BI_LSTM


In [35]:
model_bi_lstm_pred_probs = model_bi_lstm.predict(test_split['Sentence'])
model_bi_lstm_preds = tf.squeeze(tf.round(model_bi_lstm_pred_probs))
calculate_results(test_split['Label'], model_bi_lstm_preds)

{'accuracy': 90.9090909090909,
 'f1': 0.9090909090909091,
 'precision': 0.9090909090909091,
 'recall': 0.9090909090909091}

### BI_GRU

In [36]:
model_bi_gru_pred_probs = model_bi_gru.predict(test_split['Sentence'])
model_bi_gru_preds = tf.squeeze(tf.round(model_bi_gru_pred_probs))
calculate_results(test_split['Label'], model_bi_gru_preds)

{'accuracy': 81.81818181818183,
 'f1': 0.8181818181818182,
 'precision': 0.8318181818181819,
 'recall': 0.8181818181818182}